<a href="https://colab.research.google.com/github/divyeshparekh9496/Autonomous-Driving-System-for-AI-Accelerators-using-PyTorch/blob/main/Autonomous_AI_accelerator_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import psutil
!pip install dash
!pip install dash_daq
import dash
from dash import dcc, html
import dash_daq as daq
from dash.dependencies import Input, Output
import sqlite3
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import datetime

In [ ]:
# Initialize SQLite database
db_file = 'ai_accelerator.db'
if not os.path.exists(db_file):
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS accelerator_data
                 (timestamp TEXT, accelerator_type TEXT, power_usage REAL, load_distribution REAL)''')
    conn.commit()
else:
    conn = sqlite3.connect(db_file)
    c = conn.cursor()


In [ ]:

# Detect AI accelerators (CPU, GPU, etc.)
def detect_accelerators():
    accelerators = []

    # CPU
    cpu_info = {"type": "CPU", "brand": "Generic CPU", "count": psutil.cpu_count(logical=True),
                "capabilities": "Multi-threading, SSE4, AVX", "memory": psutil.virtual_memory().total}
    accelerators.append(cpu_info)

    # GPU
    try:
        import GPUtil
        gpus = GPUtil.getGPUs()
        for gpu in gpus:
            accelerators.append({
                "type": "GPU",
                "brand": gpu.name,
                "count": 1,
                "capabilities": "CUDA, Tensor cores",
                "memory": gpu.memoryTotal
            })
    except ImportError:
        pass

    return accelerators


In [ ]:

# Reinforcement Learning Model
class AIOptimizer(nn.Module):
    def __init__(self):
        super(AIOptimizer, self).__init__()
        self.fc1 = nn.Linear(2, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


In [ ]:

# Initialize model and optimizer
model = AIOptimizer()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()


In [ ]:
# Train RL Model (dummy training loop)
def train_model(state, target):
    state = torch.tensor(state, dtype=torch.float32)
    target = torch.tensor(target, dtype=torch.float32)

    optimizer.zero_grad()
    output = model(state)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()


In [ ]:
# AI Recommendations based on RL
def ai_recommendations(power, load):
    state = np.array([power, load])
    with torch.no_grad():
        recommendation = model(torch.tensor(state, dtype=torch.float32))
    return f"Recommended Power: {recommendation[0].item():.2f}, Recommended Load: {recommendation[1].item():.2f}"


In [ ]:
# Power and Load Distribution sliders
def power_management_layout():
    return html.Div([
        daq.Slider(
            id='power-slider',
            min=0,
            max=100,
            value=50,
            handleLabel={"showCurrentValue": True, "label": "Power"},
            step=1,
            marks={0: 'Low', 100: 'High'},
        ),
        html.Div(id='power-output'),
    ])

def load_distribution_layout():
    return html.Div([
        daq.Slider(
            id='load-slider',
            min=0,
            max=100,
            value=50,
            handleLabel={"showCurrentValue": True, "label": "Load"},
            step=1,
            marks={0: 'CPU', 100: 'GPU'},
        ),
        html.Div(id='load-output'),
    ])


In [20]:

# Dash Application
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Autonomous AI Accelerator Manager"),

    html.H2("Detected Accelerators"),
    html.Div(id="accelerator-info"),

    html.H2("Memory Layer & Power Optimization"),
    power_management_layout(),

    html.H2("Load Distribution"),
    load_distribution_layout(),

    html.H2("AI-Generated Recommendations"),
    html.Div(id="ai-recommendations"),

    html.H2("Real-time Monitoring"),
    dcc.Interval(id='interval-component', interval=5*1000, n_intervals=0),

    html.Div(id='live-update-text'),
])


In [21]:
# Callbacks for updating the UI based on accelerator detection and recommendations
@app.callback(
    [Output('accelerator-info', 'children'), Output('ai-recommendations', 'children')],
    [Input('interval-component', 'n_intervals'), Input('power-slider', 'value'), Input('load-slider', 'value')]
)
def update_accelerator_info(n, power, load):
    accelerators = detect_accelerators()
    accelerator_info = [f"Type: {acc['type']}, Brand: {acc['brand']}, Count: {acc['count']}, "
                        f"Capabilities: {acc['capabilities']}, Memory: {acc['memory']}" for acc in accelerators]

    recommendations = ai_recommendations(power, load)

    return html.Div([html.P(info) for info in accelerator_info]), recommendations

@app.callback(
    Output('power-output', 'children'),
    [Input('power-slider', 'value')]
)
def update_power(value):
    return f'Current Power Setting: {value}%'

@app.callback(
    Output('load-output', 'children'),
    [Input('load-slider', 'value')]
)
def update_load(value):
    return f'Current Load Distribution: {value}%'

# Real-time monitoring of power utilization and load distribution
@app.callback(
    Output('live-update-text', 'children'),
    [Input('interval-component', 'n_intervals'), Input('power-slider', 'value'), Input('load-slider', 'value')]
)
def update_metrics(n, power, load):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Simulated power and load values
    power_usage = power * 1.5
    load_distribution = load

    # Insert into the database
    c.execute("INSERT INTO accelerator_data VALUES (?, ?, ?, ?)",
              (timestamp, 'AI Accelerator', power_usage, load_distribution))
    conn.commit()

    return [f"Timestamp: {timestamp}, Power Usage: {power_usage}W, Load Distribution: {load_distribution}%"]

# Running the Dash app
if __name__ == '__main__':
    app.run_server(debug=False) # Disable debug mode

<IPython.core.display.Javascript object>